<h1 id="basics" style="font-family:verdana;"> 
    <center> Customer Life Time Value Prediction for Online Retail Dataset
    </center>
</h1>
<div style="width:100%;text-align: center;"> <img align=middle src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*Ghpzwy6mX2QGXtOO9e8KVw.jpeg" alt="CLTV" style="height:500px;margin-top:2rem;"> </div>



This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

## Main topics of the study can be seen below:

* [Aim of the study](#section-one)
* [Understanding the data](#section-two)
* [Preparation of data](#section-three)
* [Preparation of the data for CLTV](#section-four)
* [BG/NBD Model](#section-five)
* [Gamma Gamma Model](#section-six)
* [CLTV Calculation with BG/NBD & Gamma Gamma Model](#section-seven)
* [CLTV Segmentation](#section-eight)
* [Conclusion](#section-nine)


<a id="section-one"></a>
## 1. Aim of the Study

The main purpose of the study to obtain the Customer Live Time Value Prediction according to Online Retail data. This data is including two different time between 2009 - 2010 and 2010 - 2011. In this part lets explain the CLTV Prediction as much as basicly.

> [Average Transaction ($)]   X   [# of Transactions]   X   [Retention time period]

> For example, if you are a SaaS company and you sell $20 monthly subscriptions for customers who stay subscribed for an average of 18 months, your CLV is:

> $20 (subscription cost) x 12 (transactions per year) x 1.5 years = $360 CLV

> Knowing your CLV is important for business decision-making and getting a better understanding of your company’s financial health and future. 
<div style="width:100%;text-align: center;"> <img align=middle src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*rRrRZ0Yl8i2qv_HV8mibAg.jpeg" alt="CLV" style="height:300px;margin-top:1rem;"> </div>

<a id="section-two"></a>
## 2. Understanding the Data

First of all we should import the libraries that will use during the analysis and rating parts.

In [1]:
# Lets import the dataset

!pip install lifetimes
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: '%.5f' % x)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 4.4 MB/s eta 0:00:00-:--:--

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Lets import the dataset

df = pd.read_csv("Year 2010-2011.csv", encoding= 'unicode_escape')


In [3]:
# To understand the "check_df" functione can be used to decide the what should we do about the data.

def check_df(dataframe, head=5):
    print("########## Info #############")
    print(dataframe.info())
    print("########## Shape #############")
    print(dataframe.shape)
    print("########## Data Types #############")
    print(dataframe.dtypes)
    print("########## Head of Data #############")
    print(dataframe.head(head))
    print("########## Tail of Data #############")
    print(dataframe.tail(head))
    print("########## Null Values of Data #############")
    print(dataframe.isnull().sum())
    print("########## Describe of the Numerical Datas #############")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

########## Info #############
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None
########## Shape #############
(541910, 8)
########## Data Types #############
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object
########## Head of Data #############
  Invoice StockCode                          De

Before the start the analysis, according to dataset summary, dataset has 8 variables. Lets check them;

1. Invoice No: Special number of Invoice for the each Customer ID
2. StockCode: Special number for the each kind of products.
3. Description: Summary of the products that sell by the company.
4. Quantity: Sell amounts of the product for each order.
5. InvoiceDate: Date of the Invoice
6. Unit Price: Unit price for the each quantity of the product.
7. CustomerID: Special ID for the customers.
8. Country: Customers Country

According to null value of data, the data has 135080 missing value for the Customer ID. These missing values should be eleminated the data in the next chapters.

Also, according to description part for the numerical variables, we can see the "-" UnitPrice or Quantity. It is meaning some kind of orders cancelled by the customers. We should also elimanite these rows to find out the meaningful results during the analysis process.

<a id="section-three"></a>
## 3. Preparation of the Data

In this stage, If any null values are in the dataset, they will drop it from the data.

In [4]:
# dropna() command will help to drop the null values from the data.
df.shape
df.isnull().sum()
df.dropna(inplace = True)

# Lets check the data

df.describe().T



,count,mean,std,min,25%,50%,75%,max
Quantity,406830.00000,12.06128,248.69306,-80995.00000,2.00000,5.00000,12.00000,80995.00000
Price,406830.00000,3.46051,69.31508,0.00000,1.25000,1.95000,3.75000,38970.00000
Customer ID,406830.00000,15287.68416,1713.60307,12346.00000,13953.00000,15152.00000,16791.00000,18287.00000


In [19]:
# According to data, If "Invoice" include "C", it is mean these orders returned. Because of that Quantity or Price for these orders is in negative sign in the dataset.

df = df[~df["Invoice"].str.contains("C", na = False)] 

# To be sure, we have to eliminate the negative Quantity values from the data.

df = df[df["Quantity"] > 0]


In [20]:
# Before the analysis we have to consider the "Outliers" in the data. Especially, outliers should be considered in "Quantity" and "Invoice" columns.

def outlier_thresholds(dataframe, variable):
    # Function prepared to find out outliers.
    # Normally, Q1 and Q3 uses as 0.25 - 0.75 but in this project we consider as 0.01 - 0.99
    
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 + quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 + 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    # Outliers majorization function
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [7]:
# For CLTV analysis, three variables are important during the analysis process. Because of that "numerical" variables has to be reconsidered for the quartiles. In this part, two functions 
# that prepared before will be applied for the Quantity and Price variables.

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [8]:
# To make a CLTV prediction, dataset has to have "Total Price" variables. Because of that it will be obtained with Quantity x Price.

df["TotalPrice"] = df["Quantity"] * df["Price"]

<a id="section-four"></a>
## 4. Preparation of the data for CLTV

To prepare the CLTV dataframe, four metrics should be calculated by the current variables.

1. Recency: Difference between the first and last purchase of the customer. (Weekly)
2. T: The age of the customer. The other words, Analysis date - first purchase date of the customer (Weekly)
3. Frequency: It means, total number of purchase of the customer. Frequency value has to be higher than 1.
4. Monetary: Average earning per purchase.

In [9]:
# To calculate the Recency and T metrics, we should decide the analysis day. 

# Analysis day should be calculated after the last purchase days + 1 or 2 days.

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

last_day = df["InvoiceDate"].max()
today_date = dt.datetime(last_day.year, last_day.month, last_day.day + 1)

In [10]:
# CLTV Dataframe Preparation

cltv_df = df.groupby("Customer ID").agg({"InvoiceDate": [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                                         lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
                               "Invoice": lambda Invoice: Invoice.nunique(),
                                "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ["recency", "T", "frequency", "monetary"]

# Monetary: Average earning per purchase (Frequency)

cltv_df["monetary"]  = cltv_df["monetary"] / cltv_df["frequency"]

# Lets filtered the data according to Frequency > 1

cltv_df = cltv_df[cltv_df["frequency"] > 1]
   
# Recency and T values is in day format. We should convert to the weekly format.                   
                  
cltv_df["recency"] = cltv_df["recency"] / 7

cltv_df["T"] = cltv_df["T"] / 7

<a id="section-five"></a>
## 5. BG/NBD Model

>Beta Geometric / Negative Binomial Distribution known as BG-NBD Model. Also sometimes it comes up as “Buy Till You Die”. It gives us the conditional expected number of transactions in the next period. This model can answer the following questions

<div style="width:100%;text-align: center;"> <img align=middle src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*ZcERUbH36qDWniPvQGZDcQ.png" alt="BG/NBD" style="height:300px;margin-top:1rem;"> </div>



>How many transactions will be next week?
How many transactions will be in the next 3 months?
Which customers will do the most purchases in the next 2 weeks?
This model models 2 processes by using probability for predicting the expected number of transactions

>1. Transaction Process (Buy)
>2. Dropout Process (Till You Die)

In [11]:
# Here is the BetaGeoFitter functions. It is used to apply the BG-NBD Model.

bgf = BetaGeoFitter(penalizer_coef= 0.001)

# Preparation of the Model

bgf.fit(cltv_df["frequency"],
        cltv_df["recency"],
        cltv_df["T"])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.34, b: 2.47, r: 2.18>

In [12]:
# Which customers are the top purchasers in a week?

cltv_df["expected_purc_1_week"] = bgf.predict(1,cltv_df["frequency"], cltv_df["recency"],cltv_df["T"])

cltv_df.sort_values("expected_purc_1_week", ascending = False)


,recency,T,frequency,monetary,expected_purc_1_week
Customer ID,,,,,
12748.00000,53.14286,53.28571,210,154.21857,3.27758
14911.00000,53.14286,53.28571,201,692.01995,3.13848
17841.00000,53.00000,53.28571,124,330.23089,1.94766
13089.00000,52.28571,52.71429,97,606.38196,1.54350
14606.00000,53.14286,53.28571,93,130.15935,1.46929
...,...,...,...,...,...
13093.00000,13.85714,53.28571,8,979.05875,0.00088
15107.00000,8.71429,53.14286,6,53.25000,0.00058
16725.00000,9.85714,53.00000,7,111.57429,0.00039


In [13]:
# Which customers are the top purchasers in a month?

cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                                cltv_df["frequency"],
                                                cltv_df["recency"],
                                                cltv_df["T"])

cltv_df.sort_values("expected_purc_1_month", ascending = False)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month
Customer ID,,,,,,
12748.00000,53.14286,53.28571,210,154.21857,3.27758,13.07597
14911.00000,53.14286,53.28571,201,692.01995,3.13848,12.52106
17841.00000,53.00000,53.28571,124,330.23089,1.94766,7.77021
13089.00000,52.28571,52.71429,97,606.38196,1.54350,6.15763
14606.00000,53.14286,53.28571,93,130.15935,1.46929,5.86173
...,...,...,...,...,...,...
13093.00000,13.85714,53.28571,8,979.05875,0.00088,0.00351
15107.00000,8.71429,53.14286,6,53.25000,0.00058,0.00231
16725.00000,9.85714,53.00000,7,111.57429,0.00039,0.00156


In [14]:
# Which customers are the top purchasers in 3 months?

cltv_df["expected_purc_3_month"] = bgf.predict(12,
                                                cltv_df["frequency"],
                                                cltv_df["recency"],
                                                cltv_df["T"])

cltv_df.sort_values("expected_purc_3_month", ascending = False)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month
Customer ID,,,,,,,
12748.00000,53.14286,53.28571,210,154.21857,3.27758,13.07597,38.96899
14911.00000,53.14286,53.28571,201,692.01995,3.13848,12.52106,37.31520
17841.00000,53.00000,53.28571,124,330.23089,1.94766,7.77021,23.15649
13089.00000,52.28571,52.71429,97,606.38196,1.54350,6.15763,18.34959
14606.00000,53.14286,53.28571,93,130.15935,1.46929,5.86173,17.46875
...,...,...,...,...,...,...,...
13093.00000,13.85714,53.28571,8,979.05875,0.00088,0.00351,0.01046
15107.00000,8.71429,53.14286,6,53.25000,0.00058,0.00231,0.00689
16725.00000,9.85714,53.00000,7,111.57429,0.00039,0.00156,0.00464


<a id="section-six"></a>
## 6. Gamma Gamma Model

>Gamma-Gamma model presented in the same paper, adds a monetary value into the mix. It does so by assuming that the spend of an individual is right-skewed and follows a Gamma distribution. One of the parameters required to describe Gamma distribution, also varies per customer (so each customer again ends up with different propensity to spend) and it also follows a Gamma distribution. That’s why the model is called Gamma-Gamma.

<div style="width:100%;text-align: center;"> <img align=middle src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gamma_distribution_pdf.svg/1280px-Gamma_distribution_pdf.svg.png" alt="Gamma Gamma" style="height:500px;margin-top:1rem;"> </div>


In [15]:
# GammaGammaFitter is using to model the GammaGamma Model

ggf = GammaGammaFitter(penalizer_coef= 0.01)

ggf.fit(cltv_df["frequency"], cltv_df["monetary"])

# It means, calculate the expected average profit.

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"], cltv_df["monetary"])


<a id="section-seven"></a>
## 7. CLTV Calculation with BG/NBD & Gamma Gamma Model



In [16]:
# Lets calculate the CLV of the CLTV dataset with BG/NBD & Gamma Gamma Model

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time = 3, # 3 Aylık
                                   freq = "W", # T'nin frekans bilgisi
                                   discount_rate= 0.01)

cltv = cltv.reset_index()

cltv_final = cltv_df.merge(cltv, on = "Customer ID", how = "left")
cltv_final.sort_values(by = "clv", ascending = False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
1122,14646.00000,50.42857,50.57143,74,3598.86568,1.22710,4.89494,14.58355,3607.37581,55976.45267
2761,18102.00000,52.28571,52.42857,60,3871.76383,0.97211,3.87805,11.55595,3883.05702,47746.18233
843,14096.00000,13.85714,14.42857,17,3164.07000,0.73553,2.92307,8.63630,3196.92414,29351.28123
36,12415.00000,44.71429,48.14286,21,5727.56262,0.38150,1.52159,4.53161,5775.46731,27846.60435
1257,14911.00000,53.14286,53.28571,201,692.01995,3.13848,12.52106,37.31520,692.63651,27501.50434
2458,17450.00000,51.28571,52.42857,46,2876.51076,0.75048,2.99389,8.92115,2887.48512,27409.36224
874,14156.00000,51.57143,53.00000,55,2104.85309,0.88140,3.51627,10.47842,2111.58348,23543.21294
2487,17511.00000,52.85714,53.28571,31,2934.18452,0.51067,2.03724,6.07083,2950.82362,19061.27615
2075,16684.00000,50.42857,51.14286,28,2214.05821,0.47987,1.91422,5.70296,2228.00015,13519.66371
1754,16000.00000,0.00000,0.28571,3,2351.88000,0.43040,1.69720,4.94227,2497.60011,13104.82500


<a id="section-eight"></a>
## 8. CLTV Segmentation

In [17]:
# Lets add the segmentation to the dataset according to clv values of the customers.

cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels = ["D", "C", "B", "A"])

cltv_final.groupby("segment").agg({"count", "sum", "mean"})

Customer ID                            recency                          T                      frequency                monetary                        expected_purc_1_week                   expected_purc_1_month                   expected_purc_3_month                    expected_average_profit                          clv                         
              count        mean            sum   count     mean         sum count     mean         sum     count     mean   sum    count      mean          sum                count    mean       sum                 count    mean       sum                 count    mean        sum                   count      mean          sum count       mean           sum
segment                                                                                                                                                                                                                                                                                                                                                              
D               712 15558.68118 11077781.00000     712 22.08868 15727.14286   712 40.34731 28727.28571       712  3.06882  2185      712 184.01480 131018.53755                  712 0.07175  51.08693                   712 0.28577 203.47159                   712 0.84848  604.11830                     712 199.47748 142027.96929   712  144.83284  103120.98164
C               711 15305.10267 10881928.00000     711 30.83183 21921.42857   711 38.03175 27040.57143       711  4.10408  2918      711 270.59478 192392.88714                  711 0.12178  86.58599                   711 0.48498 344.82422                   711 1.43965 1023.59188                     711 288.80173 205338.03271   711  383.65972  272782.05798
B               711 15356.51758 10918484.00000     711 29.43339 20927.14286   711 34.66626 24647.71429       711  5.44726  3873      711 374.61954 266354.49072                  711 0.16340 116.17401                   711 0.65033 462.38371                   711 1.92813 1370.90151                     711 395.10185 280917.41659   711  693.69005  493213.62836
A               711 14947.98594 10628018.00000     711 31.31525 22265.14286   711 34.25497 24355.28571       711 11.34459  8066      711 660.12402 469348.17986                  711 0.27549 195.87418                   711 1.09683 779.84650                   711 3.25437 2313.85952                     711 686.25337 487926.14881   711 2239.60434 1592358.68731

In [18]:
# Lets sort the customers according to the CLTV predicted values. Who is the best and valuable customers for the client?

cltv_final.sort_values(by = "clv", ascending = False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv,segment
1122,14646.00000,50.42857,50.57143,74,3598.86568,1.22710,4.89494,14.58355,3607.37581,55976.45267,A
2761,18102.00000,52.28571,52.42857,60,3871.76383,0.97211,3.87805,11.55595,3883.05702,47746.18233,A
843,14096.00000,13.85714,14.42857,17,3164.07000,0.73553,2.92307,8.63630,3196.92414,29351.28123,A
36,12415.00000,44.71429,48.14286,21,5727.56262,0.38150,1.52159,4.53161,5775.46731,27846.60435,A
1257,14911.00000,53.14286,53.28571,201,692.01995,3.13848,12.52106,37.31520,692.63651,27501.50434,A
2458,17450.00000,51.28571,52.42857,46,2876.51076,0.75048,2.99389,8.92115,2887.48512,27409.36224,A
874,14156.00000,51.57143,53.00000,55,2104.85309,0.88140,3.51627,10.47842,2111.58348,23543.21294,A
2487,17511.00000,52.85714,53.28571,31,2934.18452,0.51067,2.03724,6.07083,2950.82362,19061.27615,A
2075,16684.00000,50.42857,51.14286,28,2214.05821,0.47987,1.91422,5.70296,2228.00015,13519.66371,A
1754,16000.00000,0.00000,0.28571,3,2351.88000,0.43040,1.69720,4.94227,2497.60011,13104.82500,A


<a id="section-nine"></a>
## 9. Conclusion

End of the CLTV prediction for Online Retail dataset between 2010 - 2011 years, we focused some kind of variables in this process stage.

1. Quantity
2. Price
3. Total Price
4. Invoice
5. InvoiceDate

The aim of the project to find out the customers behavior for the future according to their "current" behavior. To reach this aim, some kind of model has to be used during the analysis process. These models:

1. BG/NBD Model
2. Gamma Gamma Model

These models give us expected purchase and average profits according to selected future date (1 week, month, 3 months or more)

End of the analysis as we can see which customers are the potential income for the client and also which client has to be under cover to stay with the company and how can client increase the customers' purchases.

## Keep in Touch!

You can follow my the other social media adresses to see this kind of works!

1. [GitHub](https://github.com/KeskinHakan)
2. [LinkedIn](https://www.linkedin.com/in/hakan-keskin-/)
3. [Medium](https://medium.com/@hakan-keskin)
